In [ ]:
import sys
print(sys.executable)

In [2]:
!pip install langchain faiss-cpu transformers bs4 -U langchain-community


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip freeze

aiohappyeyeballs==2.5.0
aiohttp==3.11.13
aiosignal==1.3.2
annotated-types==0.7.0
anyio==4.8.0
appnope==0.1.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==3.0.0
async-lru==2.0.4
attrs==25.1.0
babel==2.17.0
beautifulsoup4==4.13.3
bleach==6.2.0
bs4==0.0.2
certifi==2025.1.31
cffi==1.17.1
charset-normalizer==3.4.1
comm==0.2.2
dataclasses-json==0.6.7
debugpy==1.8.13
decorator==5.2.1
defusedxml==0.7.1
executing==2.2.0
faiss-cpu==1.10.0
fastjsonschema==2.21.1
filelock==3.17.0
fqdn==1.5.1
frozenlist==1.5.0
fsspec==2025.2.0
greenlet==3.1.1
h11==0.14.0
httpcore==1.0.7
httpx==0.28.1
httpx-sse==0.4.0
huggingface-hub==0.29.2
idna==3.10
ipykernel==6.29.5
ipython==9.0.1
ipython_pygments_lexers==1.1.1
ipywidgets==8.1.5
isoduration==20.11.0
jedi==0.19.2
Jinja2==3.1.6
json5==0.10.0
jsonpatch==1.33
jsonpointer==3.0.0
jsonschema==4.23.0
jsonschema-specifications==2024.10.1
jupyter==1.1.1
jupyter-console==6.6.3
jupyter-events==0.12.0
jupyter-lsp==2.2.5
jupyter_client==8.6.3
jupyt

## Web-scraping information and writing to file

In [47]:
import requests
from bs4 import BeautifulSoup

URL = "https://smartasset.com/investing/stock-correlation"

r = requests.get(URL)
#print(r.content)

soup = BeautifulSoup(r.text, 'html.parser')
paragraphs = soup.find_all('p')
paragraph_texts = [p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)]

filename = "corr.txt"
with open(f"/kaggle/input/knowledge_base/{filename}", 'w', encoding = 'utf-8') as file:
  for paragraph in paragraph_texts:
    file.write(paragraph + "\n\n")


## For Kaggle

In [49]:
import requests
from bs4 import BeautifulSoup

URL ="https://smartasset.com/investing/stock-correlation"

r = requests.get(URL)
#print(r.content)

soup = BeautifulSoup(r.text, 'html.parser')
paragraphs = soup.find_all('p')
paragraph_texts = [p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)]

filename = "corr.txt"
with open(f"/kaggle/working/{filename}", 'w', encoding = 'utf-8') as file:
  for paragraph in paragraph_texts:
    file.write(paragraph + "\n\n")

## Appending to file

In [ ]:
import requests
from bs4 import BeautifulSoup

URL =input("Enter URL to scrape: ")

r = requests.get(URL)
print(r.content)

soup = BeautifulSoup(r.text, 'html.parser')
paragraphs = soup.find_all('p')
paragraph_texts = [p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)]

filename = input("Enter relevant file name: ")
with open(f"/content/{filename}", 'a', encoding = 'utf-8') as file:
  for paragraph in paragraph_texts:
    file.write(paragraph + "\n\n")

In [2]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

file_paths = [
    "Knowledge_Base/candlestick.txt",
    "Knowledge_Base/ma.txt",
    "Knowledge_Base/momentum.txt",
    "Knowledge_Base/rsi.txt",
    "Knowledge_Base/bollinger.txt",
    "Knowledge_Base/corr.txt",
    "Knowledge_Base/cumul.txt",
    "Knowledge_Base/macd.txt"
]

documents = []
for file_path in file_paths:
    loader = TextLoader(file_path)
    documents.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

ModuleNotFoundError: No module named 'langchain'

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings

# Use sentence-transformers for generating embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedding_model.embed_documents([chunk.page_content for chunk in chunks])

ModuleNotFoundError: No module named 'langchain'

In [52]:
from langchain.vectorstores import FAISS

# Use a FAISS vector database to store document embeddings
vectorstore = FAISS.from_documents(documents=chunks, embedding=embedding_model)

In [54]:
class QAPipelineWrapper:
    def __init__(self, pipeline):
        self.pipeline = pipeline
        self.task = "question-answering"

    def __call__(self, prompt, **kwargs):
        # Format the input for the QA pipeline
        question = kwargs.get("question", "What is Momentum?")  # Default question if not provided
        context = prompt  # The prompt contains the context
        # Clean the context to remove any special characters or formatting
        context = self._clean_text(context[0])
        
        # Format the input for the QA pipeline
        inputs = {
            "question": question,
            "context": context
        }
        return self.pipeline(inputs)["answer"]

    def _clean_text(self, text):
        text = text.replace("\n", " ").replace("\t", " ")
        text = "".join(char for char in text if ord(char) < 128)
        return text

In [61]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA


model_name = "google/flan-t5-large"  # You can choose other models
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set up the pipeline
hf_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,  # Max token length for output
    num_beams=5,     # Beam search for better answers, change from 3
    temperature = 0.5,
    do_sample = True,
    top_p = 0.9 #nucleus sampling
)

# Wrap the pipeline for LangChain
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Use your retriever (assuming `vectorstore` is already defined)
retriever = vectorstore.as_retriever()

# Create a RetrievalQA chain
rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",  # "map_reduce", "refine", "map_rerank", etc.
    retriever=retriever
)

question = "Can you explain momentum in absolute detail?"
response = rag_pipeline.run(question)
print(f"Q: {question}\nA: {response}")

Device set to use cuda:0


Q: Can you explain momentum in absolute detail?
A: Momentum investing is primarily a short-term strategy. It focuses on identifying stocks that are currently trending upward and buying them with the expectation that the trend will continue in the short term. As soon as the momentum starts to fade or reverse, the investor sells the stock to capture the short-term gains. To calculate a stock's momentum, you can use the following formula: Momentum = Current Price / Price X months ago


In [63]:
vectorstore.save_local("/kaggle/working/retriever_data/")
print("Vectorstore saved successfully!")


save_directory = "/kaggle/working/generative_model"
pipeline = llm.pipeline 

pipeline.model.save_pretrained(save_directory)
pipeline.tokenizer.save_pretrained(save_directory)
print("Generative model and tokenizer saved successfully!")

import json

# Save the configuration of the RAG pipeline
rag_config = {
    "chain_type": "stuff",
    "retriever_path": "/kaggle/working/retriever_data", 
    "model_path": "/kaggle/working/generative_model",  
    "tokenizer_path": "/kaggle/working/generative_model"  
}


with open("/kaggle/working/rag_config.json", "w") as f:
    json.dump(rag_config, f)

print("RAG pipeline configuration saved successfully!")


Vectorstore saved successfully!
Generative model and tokenizer saved successfully!
RAG pipeline configuration saved successfully!


In [62]:
question = "What is MACD ?"
response = rag_pipeline.run(question)
print(f"Q: {question}\nA: {response}")

Q: What is MACD ?
A: MACD full form: Moving Average Convergence Divergence and is one of the most widely used momentum indicators in technical analysis. Gerald Appel was the creator of this indicator at the end of the 1970s. By computing the distinction between two time period intervals, which are a compilation of historical time series, this indicator is used to define momentum and its directional resilience. MACD uses moving averages of two distinct time intervals (most commonly historical closing prices of ------------


In [ ]:
import json
from langchain.chains import RetrievalQA
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import FAISS

# Load the RAG configuration
with open("/kaggle/working/rag_config.json", "r") as f:
    rag_config = json.load(f)

# Load the vector store
vectorstore = FAISS.load_local(rag_config["retriever_path"])

# Load the model and tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(rag_config["model_path"])
tokenizer = AutoTokenizer.from_pretrained(rag_config["tokenizer_path"])

# Create the QA pipeline
hf_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Wrap it in LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Initialize the retriever
retriever = vectorstore.as_retriever()

# Recreate the RAG pipeline
rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type=rag_config["chain_type"],  # Use the saved chain type
    retriever=retriever
)

# Ask a question
response = rag_pipeline.run("What is Momentum?")
print(response)